In [1]:
""" Libraries """
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.wkt import loads
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
""" Input a dataframe with two columns (linestring geometry attribute in string form, state in int form) and a name string, output the visualization """


def draw_lines(df, name):
    list_state = df["state"].tolist()
    list_geometry = df["geometry"].tolist()
    list_lines = [loads(geometry) for geometry in list_geometry]
    list_color = ["black" if state == 0 else "orange" for state in list_state]
    del list_state, list_geometry

    fig = plt.figure(figsize=(6, 8))
    ax = fig.add_subplot(111, projection='3d')
    for lines, color in tqdm(zip(list_lines, list_color)):
        x, y, z = zip(*list(lines.coords))
        ax.plot(x, y, z, c=color)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('T', labelpad=-30)
    ax.set_xlim(0, 21664)
    ax.set_ylim(0, 21352)
    ax.set_zlim(0, 86400)
    ax.set_proj_type('ortho')
    ax.set_box_aspect([1, 1, 0.75])
    legend_elements = [
        Line2D([0], [0], color='black', lw=4, label='non-working'),
        Line2D([0], [0], color='orange', lw=4, label='working')
    ]
    ax.legend(handles=legend_elements)
    plt.show()
    fig.savefig("figs/{}.png".format(name), bbox_inches='tight')

In [3]:
def draw_points(df, name, size):
    list_state = df["state"].tolist()
    list_geometry = df["geometry"].tolist()
    list_lines = [loads(geometry) for geometry in list_geometry]
    list_color = ["black" if state == 0 else "orange" for state in list_state]

    fig = plt.figure(figsize=(6, 8))
    ax = fig.add_subplot(111, projection='3d')

    all_points = []
    all_colors = []
    for line, color in zip(list_lines, list_color):
        points = np.array(line.coords)
        num_points = len(points)
        all_points.append(points)
        all_colors.extend([color] * num_points)

    all_points = np.concatenate(all_points)
    xs, ys, zs = all_points[:, 0], all_points[:, 1], all_points[:, 2]

    ax.scatter(xs, ys, zs, c=all_colors, marker='o', s=size)  # Reduce marker size here

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('T', labelpad=-30)
    ax.set_xlim(0, 21664)
    ax.set_ylim(0, 21352)
    ax.set_zlim(0, 86400)
    ax.set_proj_type('ortho')
    ax.set_box_aspect([1, 1, 0.75])

    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label='non-working', markerfacecolor='black', markersize=4),
        plt.Line2D([0], [0], marker='o', color='w', label='working', markerfacecolor='orange', markersize=4)
    ]
    ax.legend(handles=legend_elements)

    plt.show()
    fig.savefig("figs/{}.png".format(name), bbox_inches='tight')

In [4]:
""" Input the two coners of the box and output the box for visualization """


def box_from_corners(corner1, corner2):
    # Create vertices from corners
    vertices = [
        [corner1[0], corner1[1], corner1[2]],  # Bottom front left
        [corner2[0], corner1[1], corner1[2]],  # Bottom front right
        [corner2[0], corner2[1], corner1[2]],  # Bottom back right
        [corner1[0], corner2[1], corner1[2]],  # Bottom back left
        [corner1[0], corner1[1], corner2[2]],  # Top front left
        [corner2[0], corner1[1], corner2[2]],  # Top front right
        [corner2[0], corner2[1], corner2[2]],  # Top back right
        [corner1[0], corner2[1], corner2[2]]  # Top back left
    ]

    # Create faces using vertices
    box = [
        [vertices[0], vertices[1], vertices[2], vertices[3]],  # Bottom face
        [vertices[4], vertices[5], vertices[6], vertices[7]],  # Top face
        [vertices[0], vertices[3], vertices[7], vertices[4]],  # Left face
        [vertices[1], vertices[2], vertices[6], vertices[5]],  # Right face
        [vertices[2], vertices[3], vertices[7], vertices[6]],  # Back face
        [vertices[0], vertices[1], vertices[5], vertices[4]]  # Front face
    ]

    return box

In [5]:
""" Input the shape of the box, output the visualization """


def draw_box(shape, name):
    box = box_from_corners(shape["min"], shape["max"])
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.add_collection3d(Poly3DCollection(box, facecolors='red', linewidths=1, edgecolors='r', alpha=0.1))
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('T', labelpad=-15)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_zlim(0, 1)
    ax.set_proj_type('ortho')
    ax.set_box_aspect([1, 1, 0.75])
    plt.show()
    fig.savefig("figs/{}.pdf".format(name), bbox_inches='tight')

In [6]:
""" Input the times for executions, the timing list, the sql expression and the curser, modify the timing list and the curser, output the records """


def experiment_execution(times, timings, sql, cur, flag):
    for _ in tqdm(range(times)):
        time_start = time.time()
        if flag:
            cur.execute(sql)
            records = cur.fetchall()
            time_end = time.time()
        else:
            cur.execute(sql)
            time_end = time.time()
            records = cur.fetchall()
        timings.append(time_end - time_start)
    return records

In [7]:
""" Input a list of 3d points, output a linestring geometry in string form """


def get_linestring(points):
    points_str = ['{} {} {}'.format(*point) for point in points]
    points_string = ', '.join(points_str)
    return 'LINESTRING Z ({})'.format(points_string)

In [8]:
""" The custom serializer function, used for JSON file exchange """


def json_numpy_serializer(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

In [9]:
""" Encode 2d row curve """


def encode_row(x, y, depth):
    row = x * (2 ** depth) + y
    return row

In [10]:
""" Decode 2d row curve """


def decode_row(row, depth):
    x, y = divmod(row, 2 ** depth)
    return x, y

In [11]:
""" Encode 2d morton curve """


def encode_morton_2d(x, y, depth):
    x_b = bin(x)[2:].zfill(depth)
    y_b = bin(y)[2:].zfill(depth)
    m_b = ""
    for i in range(depth):
        m_b += x_b[i] + y_b[i]
    return int(m_b, 2)

In [12]:
""" Decode 2d morton curve """


def decode_morton_2d(morton, depth):
    m_b = bin(morton)[2:].zfill(depth * 2)
    x_b, y_b = "", ""
    for i in range(depth):
        x_b += m_b[2 * i]
        y_b += m_b[2 * i + 1]
    return int(x_b, 2), int(y_b, 2)

In [13]:
""" Encode 3d morton curve """


def encode_morton(x, y, t, depth):
    x_b = bin(x)[2:].zfill(depth)
    y_b = bin(y)[2:].zfill(depth)
    t_b = bin(t)[2:].zfill(depth)
    m_b = ""
    for i in range(depth):
        m_b += x_b[i] + y_b[i] + t_b[i]
    return int(m_b, 2)

In [14]:
""" Decode 3d morton curve """


def decode_morton(morton, depth):
    m_b = bin(morton)[2:].zfill(depth * 3)
    x_b, y_b, t_b = "", "", ""
    for i in range(depth):
        x_b += m_b[3 * i]
        y_b += m_b[3 * i + 1]
        t_b += m_b[3 * i + 2]
    return int(x_b, 2), int(y_b, 2), int(t_b, 2)

In [15]:
""" Convert a point to cell by flooring its coordinates """


def get_cell(point):
    return tuple(map(int, np.floor(point)))

In [16]:
""" Calculate the next point intersected with the boundary based on the direction, avoiding overshooting past the containing the end point """


def next_point(curr_p, direction, end_p, t):
    for i in range(len(direction)):
        if direction[i] != 0:
            if direction[i] > 0:
                target_boundary = np.floor(curr_p[i]) + 1
            else:
                target_boundary = np.ceil(curr_p[i]) - 1

            proposed_t = (target_boundary - curr_p[i]) / direction[i]
            next_p_proposal = curr_p + direction * proposed_t
            if ((direction[i] > 0 and next_p_proposal[i] > end_p[i]) or
                    (direction[i] < 0 and next_p_proposal[i] < end_p[i])):
                t[i] = np.inf
            else:
                t[i] = proposed_t
        else:
            t[i] = np.inf

    t_min = np.min(t)
    if t_min == np.inf:
        return end_p
    return curr_p + direction * t_min

In [17]:
""" Input two extremes of a unit points, output the intermediate points """


def intermediate_points(start_p, end_p):
    direction = end_p - start_p
    t = np.zeros_like(direction)

    points = []

    curr_p = start_p.copy()
    while not np.array_equal(get_cell(curr_p), get_cell(end_p)):
        next_p = next_point(curr_p, direction, end_p, t)
        if np.array_equal(next_p, end_p) or np.array_equal(next_p, curr_p):
            break
        points.append(next_p)
        curr_p = next_p

    # Check if points list is empty or the last point is not the end point, and add end point if necessary.
    if len(points) == 0 or not np.array_equal(points[-1], end_p):
        pass

    return np.array(points)

In [18]:
""" Node for Octree """


class nDNode:
    def __init__(self, depth_tree, depth_node, morton, size):
        self.depth_tree = depth_tree
        self.depth_node = depth_node
        self.morton = morton
        self.left = morton * 8 ** (depth_tree - depth_node)
        self.right = (morton + 1) * 8 ** (depth_tree - depth_node) - 1
        self.size = size
        corner_min = decode_morton(self.left, depth_tree + 1)
        corner_max = decode_morton(self.right, depth_tree + 1)
        corner_max = tuple(component + 1 for component in corner_max)
        self.box = {"min": corner_min, "max": corner_max}
        self.child = list()

    def __str__(self):
        return "depth_tree:{},\t depth_node:{},\t morton:{},\t left:{},\t right:{},\t size:{}".format(self.depth_tree, self.depth_node, self.morton, self.left, self.right, self.size)

In [19]:
""" Determine the point and box relationship """


def point_box_relation(point, box):
    x, y, z = point
    xmin, ymin, zmin = box['min']
    xmax, ymax, zmax = box['max']

    # Check if the point is inside the box
    if xmin < x < xmax and ymin < y < ymax and zmin < z < zmax:
        return 1

    # Check if the point is on the box
    if xmin <= x <= xmax and ymin <= y <= ymax and zmin <= z <= zmax:
        on_x_min_max = (x == xmin or x == xmax)
        on_y_min_max = (y == ymin or y == ymax)
        on_z_min_max = (z == zmin or z == zmax)
        count_true = sum([on_x_min_max, on_y_min_max, on_z_min_max])

        if count_true == 3:
            return 0
        elif count_true == 2:
            return 0
        elif count_true == 1:
            return 0

    # If none of the above, the point must be outside the box
    return -1

In [20]:
""" Check the box to box relationship """


def check_relation(relations):
    case_in = 1 in relations
    case_on = 0 in relations
    case_ex = -1 in relations

    if case_in and case_on and case_ex:
        return "intersected"
    elif case_in and case_on and not case_ex:
        return "inside"
    elif case_in and not case_on and case_ex:
        return "intersected"
    elif case_in and not case_on and not case_ex:
        return "inside"
    elif not case_in and case_on and case_ex:
        return "outside"
    elif not case_in and case_on and not case_ex:
        return None
    elif not case_in and not case_on and case_ex:
        return "outside"
    elif not case_in and not case_on and not case_ex:
        return None

In [21]:
""" Determine the box and box relationship """


def tree_shape_relation(treeBox, shapeBox):
    # Generate all corners of treeBox and shapeBox
    tree_corners = [
        [x, y, z]
        for x in [treeBox['min'][0], treeBox['max'][0]]
        for y in [treeBox['min'][1], treeBox['max'][1]]
        for z in [treeBox['min'][2], treeBox['max'][2]]
    ]
    shape_corners = [
        [x, y, z]
        for x in [shapeBox['min'][0], shapeBox['max'][0]]
        for y in [shapeBox['min'][1], shapeBox['max'][1]]
        for z in [shapeBox['min'][2], shapeBox['max'][2]]
    ]

    tree_to_shape = [point_box_relation(corner, shapeBox) for corner in tree_corners]
    shape_to_tree = [point_box_relation(corner, treeBox) for corner in shape_corners]

    if check_relation(shape_to_tree) == "inside" and check_relation(tree_to_shape) == "inside":
        return None
    elif check_relation(shape_to_tree) == "inside" and check_relation(tree_to_shape) == "intersected":
        return "intersected"
    elif check_relation(shape_to_tree) == "inside" and check_relation(tree_to_shape) == "outside":
        return "contained"
    elif check_relation(shape_to_tree) == "intersected" and check_relation(tree_to_shape) == "inside":
        return "intersected"
    elif check_relation(shape_to_tree) == "intersected" and check_relation(tree_to_shape) == "intersected":
        return "intersected"
    elif check_relation(shape_to_tree) == "intersected" and check_relation(tree_to_shape) == "outside":
        return "intersected"
    elif check_relation(shape_to_tree) == "outside" and check_relation(tree_to_shape) == "inside":
        return "inside"
    elif check_relation(shape_to_tree) == "outside" and check_relation(tree_to_shape) == "intersected":
        return "intersected"
    else:
        # elif check_relation(shape_to_tree) == "outside" and check_relation(tree_to_shape) == "outside":
        return "outside"

In [22]:
""" Class for Octree """


class nDTree:
    def __init__(self, depth_tree, dist, threshold_size):
        # Initialize the root node
        self.root = nDNode(depth_tree, 0, dist[0][0][0], dist[0][0][1])
        # Consider root node as parent
        parents = [self.root]
        # For each level of the tree
        for i in range(1, depth_tree + 1):
            # Map the parents and their children
            children = dist[i]
            j = 0
            new_parents = []
            # For each parent
            for parent in parents:
                # Size of parent is so small, no need to refine
                if parent.size <= threshold_size:
                    continue
                # Not all the children are traversed
                while j < len(children):
                    # If this child belong to this parent
                    if parent.left <= children[j][0] * 8 ** (depth_tree - i) <= parent.right:
                        # If the parent does not have child currently
                        if parent.child is None:
                            parent.child = []
                        else:
                            pass
                        # Map the parent and the child
                        parent.child.append(nDNode(depth_tree, i, children[j][0], children[j][1]))
                        j = j + 1
                    else:
                        break
            # For the next depth
            for parent in parents:
                if parent.child is not None:
                    new_parents.extend(parent.child)
            parents = new_parents

    def shapeQuery(self, queryShape, depth_query, tree_shape_relation):
        ranges = []
        parents = [self.root]

        while parents:
            parent = parents.pop()
            relation = tree_shape_relation(parent.box, queryShape)
            cant_be_refined = parent.depth_node >= depth_query or len(parent.child) == 0

            if relation == "contained":
                if cant_be_refined:
                    print("contained, stop", parent.left, parent.right)
                    ranges.append((parent.left, parent.right, 0, parent.size))
                else:
                    print("contained, continue", parent.left, parent.right)
                    parents.extend(reversed(parent.child))
            elif relation == "intersected":
                if cant_be_refined:
                    print("intersected, stop", parent.left, parent.right)
                    ranges.append((parent.left, parent.right, 0, parent.size))
                else:
                    print("intersected, continue", parent.left, parent.right)
                    parents.extend(reversed(parent.child))
            elif relation == "inside":
                print("inside", parent.left, parent.right)
                ranges.append((parent.left, parent.right, 1, parent.size))
            else:
                print("outside", parent.left, parent.right)
                ranges.append((parent.left, parent.right, -1, parent.size))

        return ranges

In [23]:
""" Class for Range """


class Range:
    def __init__(self, left, right, tag, size):
        self.left = left
        self.right = right
        self.tag = tag
        self.size = size
        self.next = None

    def __str__(self):
        if self.tag == 1:
            tag_str = "inside"
        elif self.tag == 0:
            tag_str = "intersected"
        else:  # self.tag== -1
            tag_str = "outside"
        return "range:[{},{}],\t size:{},\t tag:{}".format(self.left, self.right, self.size, tag_str)

    def __repr__(self):
        return str(self)

In [24]:
""" Class for Ranges """


class Ranges:
    def __init__(self, values, weight=5):
        self.head = None
        self.tail = None
        for val in values:
            self.insertAtEnd(val[0], val[1], val[2], val[3])
        self.weight = weight

    def insertAtEnd(self, left, right, tag, size):
        next_range = Range(left, right, tag, size)
        if self.head is None:
            self.head = next_range
            self.tail = next_range
        else:
            if self.tail.tag == next_range.tag:
                self.tail.right = next_range.right
                self.tail.size = self.tail.size + next_range.size
            else:
                self.tail.next = next_range
                self.tail = next_range

    def calculateCost(self, left_range, right_range):
        cost = 0
        if left_range.tag == right_range.tag:
            return 0
        else:
            if left_range.tag == 1 and right_range.tag == 0:
                return left_range.size
            elif left_range.tag == 1 and right_range.tag == -1:
                return left_range.size + right_range.size * self.weight
            elif left_range.tag == 0 and right_range.tag == 1:
                return right_range.size
            elif left_range.tag == 0 and right_range.tag == -1:
                return right_range.size * self.weight
            elif left_range.tag == -1 and right_range.tag == 1:
                return left_range.size * self.weight + right_range.size
            # left_range.tag == -1 and right_range.tag == 0
            else:
                return left_range.size * self.weight

    def mergeRange(self):
        if not self.head or not self.head.next:
            return None

        opt_left_range = None
        opt_right_range = None
        costMin = float("inf")

        cur_left_range = self.head
        while cur_left_range and cur_left_range.next:
            cur_right_range = cur_left_range.next
            costCur = self.calculateCost(cur_left_range, cur_right_range)
            if costCur < costMin:
                costMin = costCur
                opt_left_range = cur_left_range
                opt_right_range = cur_right_range
            cur_left_range = cur_left_range.next

        if opt_left_range and opt_right_range:
            opt_left_range.right = opt_right_range.right
            opt_left_range.tag = 0
            opt_left_range.size = opt_left_range.size + opt_right_range.size
            opt_left_range.next = opt_right_range.next
            if opt_right_range == self.tail:
                self.tail = opt_left_range
            return (opt_left_range.left, opt_left_range.right)

    def rangeList(self):
        ranges_list = []
        cur_range = self.head
        while cur_range:
            ranges_list.append((cur_range.left, cur_range.right, cur_range.tag, cur_range.size))
            cur_range = cur_range.next
        return ranges_list

    def __str__(self):
        ranges_str = []
        cur_range = self.head
        while cur_range:
            ranges_str.append(str(cur_range))
            cur_range = cur_range.next
        return "\n".join(ranges_str)

    def __repr__(self):
        return str(self)